In [15]:
# in this module we will pass our features and labels to the classifier we will build 
# we haven't right now quite defined our features and labels , which we will do in this one

# https://www.youtube.com/watch?v=r4mwkS2T9aI&list=PLQVvvaa0QuDfKTOs3Keq_kaG2P55YRn5v&index=4

# we are also going to import numpy library because python does not allow using arrays 
# plus we are going to import sklearn which we will be using for scaling
# in terms of scaling , our goal is to keep the features' value b/w -1 to +1
# this helps with accuracy and processing speed
# there are instances where people choose not to do pre processing .. that will be explained later 
# we are also using model_selection to create our training and testing samples
# it's just a nice way to split the data (it shuffles your data so that you don't have biased data)

# we are also bringing in svm
# we are not doing svm model right now but regression is also done using svm 
# so we'll be discussing this as an example here because once we finish regression we won't come back


import pandas as pd
import quandl 
import math
import numpy as np
from sklearn import preprocessing, model_selection, svm
from sklearn.linear_model import LinearRegression

df=quandl.get('WIKI/GOOGL',api_key='x1sxcyxyfLTV1Ws2tT7y')
df=df[['Adj. Open','Adj. High','Adj. Low','Adj. Close','Adj. Volume']]
df['HL_PCT']=(df['Adj. High']-df['Adj. Low'])/df['Adj. Low']*100.0
df['PCT_change']=(df['Adj. Close']-df['Adj. Open'])/df['Adj. Open']*100.0

df=df[['Adj. Close','HL_PCT','PCT_change','Adj. Volume']]

forecast_col='Adj. Close'
df.fillna(-9999, inplace=True)

forecast_out=int(math.ceil(0.01*len(df)))

print(forecast_out)

df['label']=df[forecast_col].shift(-forecast_out)
df.dropna(inplace=True)
print(df.head())

35
            Adj. Close    HL_PCT  PCT_change  Adj. Volume      label
Date                                                                
2004-08-19   50.322842  8.441017    0.324968   44659000.0  69.078238
2004-08-20   54.322689  8.537313    7.227007   22834300.0  67.839414
2004-08-23   54.869377  4.062357   -1.227880   18256100.0  68.912727
2004-08-24   52.597363  7.753210   -5.726357   15247300.0  70.668146
2004-08-25   53.164113  3.966115    1.183658    9188600.0  71.219849


In [16]:
# the first thing we need to do is define x and y 
# generally X is features and y is labels and then we will scale X and then we will redefine x according to forecast_out
# and then we will redefine y

# drop() will put all the cols of df in X except labels and df.drop() returns a new df & that is converted to an array using np
# the integer argument 1 means axis=1 which means coloumn wise dropping 

# we are scaling all our features together (feature scaling and normalization) before feeding the data to the classifier
# but in future if we add some feature to it, it needs to be scaled TOGETHER with all the other data 
# which means you will have to include it in your training data 
# this preprocessing.scale() helps in trainig in testing 
# but it may add to the processing time for projects like high frequency trading where you may skip this step

# now we want only those x's for which there is a value of y
# and this is so beacuse of shifting (forecast_out)
# example :-  a[:-2]   # everything except the last two items 
# https://stackoverflow.com/questions/45034747/meaning-of-x-x-forecast-out1
# https://stackoverflow.com/questions/509211/understanding-slice-notation
# beacuse we have used number of days = forecast_out to make our label cloumn and by that much only we have shifted
# if forecast_out =10 , that means today we are predicting value of close for the 10 th day from today 
# and hence we shifted the rows by that number

# inplace: It is a boolean which makes the changes in data frame itself if True.
# df.dropna(inplace=True)    // Keep the DataFrame with valid entries in the same variable.

print(df.head())
X=np.array(df.drop(['label'],1))
print(X)
y=np.array(df['label'])
print(y)

X=preprocessing.scale(X)

# X=X[:-forecast_out+1]
# df.dropna(inplace =True)

y=np.array(df['label'])
print(len(X),len(y))


# these are not the correct lengths(len of x should be equal to len of y)
# this is caused by the shifting done using the forecasr_out value in the code line :- "X=X[:-forecast_out+1]"
# we were doing this shifting because we did not want to have rows of X where there wasn't a value of y
# but that is not needed because we have used the command :- "df.dropna(inplace=True)" in the previous segment of the code 
# therefore we'll comment out that forecast shiftinf code line
# and also the commented command after the forecast_out one is also not needed for the same reason 

            Adj. Close    HL_PCT  PCT_change  Adj. Volume      label
Date                                                                
2004-08-19   50.322842  8.441017    0.324968   44659000.0  69.078238
2004-08-20   54.322689  8.537313    7.227007   22834300.0  67.839414
2004-08-23   54.869377  4.062357   -1.227880   18256100.0  68.912727
2004-08-24   52.597363  7.753210   -5.726357   15247300.0  70.668146
2004-08-25   53.164113  3.966115    1.183658    9188600.0  71.219849
[[ 5.03228418e+01  8.44101709e+00  3.24967503e-01  4.46590000e+07]
 [ 5.43226889e+01  8.53731343e+00  7.22700723e+00  2.28343000e+07]
 [ 5.48693765e+01  4.06235672e+00 -1.22788010e+00  1.82561000e+07]
 ...
 [ 1.18159000e+03  1.54700007e+00  4.76194525e-01  2.77496700e+06]
 [ 1.11920000e+03  1.81160398e+00 -7.29098295e-01  5.79888000e+06]
 [ 1.06876000e+03  5.51223574e+00 -2.89384977e+00  3.74246900e+06]]
[  69.0782379    67.83941377   68.91272699 ... 1026.55       1054.09
 1006.94      ]
3389 3389


In [17]:
# now we are ready for creating our training and testing sets
# for this we will use cross validation which shuffles the data and splits it
# 0.2 means that 20% of the data is for testing 
# and then we will use X_train and y_train to fit the classifier and the other two for later testing
# we will use linear regression classifier and fit the data 
# and then before using it in real world we will test it and check the accuracy
# and we get 97% accuracy when shifted with 1% of the days which is predicting  35 days in advance
# but this is linear regression which is based on squared error so this isn't "accurate" accurate




X_train,X_test,y_train,y_test=model_selection.train_test_split(X,y,test_size=0.2)
clf=LinearRegression()
clf.fit(X_train,y_train)
accuracy=clf.score(X_test,y_test)
print(accuracy)

0.9788941692546236


In [20]:
# now we are trying SVM
# for this accuracy got lowered -79%
# by default the kernel here is linear , if we change the kernel to "poly" accuracy decreased further
# therefore we are not going to  use SVM in this case



clf=svm.SVR()
clf.fit(X_train,y_train)
accuracy=clf.score(X_test,y_test)
print(accuracy)




clf=svm.SVR(kernel='poly')
clf.fit(X_train,y_train)
accuracy=clf.score(X_test,y_test)
print(accuracy)

0.7908782434138807
0.6653956184480256
